# AOE2
The following kernel process a subset of Age of Empires DE games

In [1]:
# Basic setup stuff
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        display(os.path.join(dirname, filename))

plt.rcParams['figure.figsize'] = (12.0, 8.0)  # default figure size

Read both the matches and match_players data to show the type of data that the files store.
* **matches.csv**: It contains information about the matches play, one by row. The match is identified by the _token_ field
* **match_players.csv**: It contains information about the players that where involved in every match. The _token_ identifies the player and the _match_ field is the foreign key of _matches.csv_

In [2]:
matches = pd.read_csv("matches.csv")
display(matches.info())
matches.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3153767 entries, 0 to 3153766
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   token           object 
 1   winning_team    int64  
 2   mirror          bool   
 3   ladder          object 
 4   patch           int64  
 5   average_rating  float64
 6   map             object 
 7   map_size        object 
 8   num_players     int64  
 9   server          object 
 10  duration        object 
dtypes: bool(1), float64(1), int64(3), object(6)
memory usage: 243.6+ MB


None

,token,winning_team,mirror,ladder,patch,average_rating,map,map_size,num_players,server,duration
0,OFb9Dkk3adCN5GLV,2,False,RM_1v1,40874,1100.0,arabia,tiny,2,ukwest,0:32:00
1,hKNeRYrz0ecdBMFO,1,False,RM_1v1,37906,NaN,arabia,tiny,2,ukwest,0:41:45
2,90ZCh6wfUVc3ujLo,2,False,RM_1v1,41855,1127.0,arabia,tiny,2,ukwest,0:16:59
3,47e37f92a0551b4d,1,False,RM_1v1,36906,1203.0,arabia,tiny,2,ukwest,0:30:17
4,zq3ILuaPrhdhgGUM,1,False,RM_1v1,40220,1059.0,valley,tiny,2,westus2,0:14:44


In [3]:
players = pd.read_csv("match_players.csv")
display(players.info())
players.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9732500 entries, 0 to 9732499
Data columns (total 7 columns):
 #   Column  Dtype  
---  ------  -----  
 0   token   object 
 1   match   object 
 2   rating  float64
 3   color   object 
 4   civ     object 
 5   team    int64  
 6   winner  bool   
dtypes: bool(1), float64(1), int64(1), object(4)
memory usage: 454.8+ MB


None

,token,match,rating,color,civ,team,winner
0,BsnDkR5pb9DVCbIt,9YDIBsStGjrIuGbW,952.0,Blue,Tatars,2,True
1,88224b59212f259b,3930e585b343d9b2,1395.0,Green,Aztecs,2,False
2,2RvyZPW1f9y4jP7M,6zAtwTFqh8OIJJuX,1330.0,Blue,Khmer,2,False
3,Ck4C5MvBqlXJLSqx,xZTyZZokUTv5Cg94,1728.0,Red,Franks,1,True
4,97991ac787bb23e3,b638b4ef08b2df96,NaN,Blue,Persians,2,True


Point 1: Games that took more than 2 hour per server

In [7]:
matches["duration"] = pd.to_timedelta(matches["duration"])
huge_duration_matches = matches[(matches["average_rating"] > 2000) & (matches["duration"] > pd.to_timedelta("02:00:00"))]
long_games = huge_duration_matches.groupby(["server"])["token"].count().reset_index().rename(columns={"token": "counts"})
print(long_games[["server", "counts"]])

               server  counts
0  australiasoutheast       8
1         brazilsouth     330
2              eastus     101
3        koreacentral       9
4       southeastasia      49
5              ukwest     246
6          westeurope       7
7           westindia      42
8             westus2      23


Point 2: Games where the player with less rating (30% difference) won the game in 1v1 games (rating > 1000)

In [ ]:
result = 0
for idx,group in players.groupby("match"):
    # Filter games with more than two players
    if len(group) == 2:
        # Check if the player with lower rating won
        conditionMet = False
        rating_diff = 0
        if group.iloc[0]["rating"] < group.iloc[1]["rating"] and group.iloc[0]["winner"] == True and group.iloc[0]["rating"] > 1000:
            rating_diff = (group.iloc[1]["rating"] - group.iloc[0]["rating"]) / group.iloc[0]["rating"] * 100
            conditionMet = True
        elif group.iloc[0]["rating"] < group.iloc[1]["rating"] and group.iloc[0]["winner"] == True and group.iloc[1]["rating"] > 1000: 
            conditionMet = True
            rating_diff = (group.iloc[0]["rating"] - group.iloc[1]["rating"]) / group.iloc[1]["rating"] * 100
            
        # Compare the rating, and only show those ones where the rating pct is above 30%
        if conditionMet and rating_diff > 30:
            result += 1

result

Join both datasets to be able to do complete points 3 and 4

In [5]:
# Drop unused fields to reduce the memory footprint of the simulation
matches.pop("map_size")
matches.pop("num_players")
matches.pop("average_rating")
matches.pop("winning_team")
matches.pop("patch")
players.pop("token")
players.pop("color")

join_matches = pd.merge(players, matches, left_on="match", right_on="token")
display(join_matches.info())
print(join_matches.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9732500 entries, 0 to 9732499
Data columns (total 11 columns):
 #   Column    Dtype  
---  ------    -----  
 0   match     object 
 1   rating    float64
 2   civ       object 
 3   team      int64  
 4   winner    bool   
 5   token     object 
 6   mirror    bool   
 7   ladder    object 
 8   map       object 
 9   server    object 
 10  duration  object 
dtypes: bool(2), float64(1), int64(1), object(7)
memory usage: 761.1+ MB


None

              match  rating      civ  team  winner             token  mirror  \
0  9YDIBsStGjrIuGbW   952.0   Tatars     2    True  9YDIBsStGjrIuGbW   False   
1  9YDIBsStGjrIuGbW   947.0  Teutons     1   False  9YDIBsStGjrIuGbW   False   
2  3930e585b343d9b2  1395.0   Aztecs     2   False  3930e585b343d9b2   False   
3  3930e585b343d9b2  1203.0  Mongols     2   False  3930e585b343d9b2   False   
4  3930e585b343d9b2  1273.0    Khmer     1    True  3930e585b343d9b2   False   

    ladder        map       server duration  
0   RM_1v1     arabia       ukwest  0:30:57  
1   RM_1v1     arabia       ukwest  0:30:57  
2  RM_TEAM  lombardia  brazilsouth  0:34:26  
3  RM_TEAM  lombardia  brazilsouth  0:34:26  
4  RM_TEAM  lombardia  brazilsouth  0:34:26  


In [17]:
print(len(join_matches))

9732500


Point 3: Civ win rate in 1v1 games in map arena

In [32]:
matches_short = matches
players_short = players

matches_1v1_no_mirror = matches_short[(matches_short["ladder"] == "RM_1v1") & (matches_short["mirror"] == False) & (matches_short["map"] == "arena")]
print(f"Len previo a mergear: {len(matches_1v1_no_mirror)}")

join_matches_1v1_no_mirror = pd.merge(players_short, matches_1v1_no_mirror, left_on="match", right_on="token")
print(f"Len en el join: {len(join_matches_1v1_no_mirror)}")
print(f"Join: {join_matches_1v1_no_mirror}")

win_per_civ = join_matches_1v1_no_mirror.groupby("civ")["winner"].mean().reset_index().rename(columns={"winner": "win_rate"}).sort_values("win_rate", ascending=False)
win_per_civ["win_rate"] = win_per_civ["win_rate"] * 100
print(win_per_civ[["civ", "win_rate"]])


Len previo a mergear: 240815
Len en el join: 481630
Join:                    match  rating         civ  team  winner             token  \
0       d1fe9df08a72f39c   828.0       Turks     1    True  d1fe9df08a72f39c   
1       d1fe9df08a72f39c     NaN    Japanese     2   False  d1fe9df08a72f39c   
2       320de965154e05ac  1120.0    Italians     2    True  320de965154e05ac   
3       320de965154e05ac  1075.0    Japanese     1   False  320de965154e05ac   
4       FcINRAJzqAnParaQ   946.0      Cumans     1    True  FcINRAJzqAnParaQ   
...                  ...     ...         ...   ...     ...               ...   
481625  BYyk4ToasDHKhSvy   713.0      Aztecs     2   False  BYyk4ToasDHKhSvy   
481626  ImBjryDyMoWFwC1W   995.0  Vietnamese     1    True  ImBjryDyMoWFwC1W   
481627  ImBjryDyMoWFwC1W   860.0      Franks     2   False  ImBjryDyMoWFwC1W   
481628  dgxRINZIh0WFlZRl  1438.0     Britons     1    True  dgxRINZIh0WFlZRl   
481629  dgxRINZIh0WFlZRl  1452.0  Bulgarians     2   False  dg

Point 4: Usage rate of every civ used by pro players (rating > 2000) in team matches in map islands

In [14]:
matches_1v1_pros = matches[(matches["ladder"] == "RM_TEAM") & (matches["map"] == "islands")]
print(len(matches_1v1_pros))

players_1v1_pros = players[(players["rating"] > 2000)]

join_matches_1v1_pros = pd.merge(players_1v1_pros, matches_1v1_pros, left_on="match", right_on="token")
print(len(join_matches_1v1_pros))

top5_civs = join_matches_1v1_pros.groupby("civ")["token"].count().reset_index().rename(columns={"token": "counts"})
top5_civs = top5_civs[["civ", "counts"]].sort_values(by='counts', ascending=False)
print(top5_civs[["civ", "counts"]])

1675
2419
            civ  counts
34      Vikings     627
15     Italians     415
26   Portuguese     350
16     Japanese     143
27     Saracens     142
21        Malay     141
5    Byzantines     131
25     Persians      65
18      Koreans      57
29      Spanish      29
1       Berbers      27
19  Lithuanians      26
2       Britons      24
24      Mongols      18
6         Celts      16
22      Malians      14
3    Bulgarians      14
32        Turks      14
9    Ethiopians      14
20      Magyars      13
13        Incas      13
12         Huns      12
8        Cumans      12
11        Goths      11
10       Franks      11
28        Slavs      11
31      Teutons      10
17        Khmer      10
14      Indians      10
7       Chinese      10
33   Vietnamese       8
0        Aztecs       8
23       Mayans       6
30       Tatars       4
4       Burmese       3
